<a href="https://colab.research.google.com/github/psyuktha/DeviVerse/blob/main/DeviVerse_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeviVerse

DeviVerse is a project focused on fine-tuning a large language model (LLM) to delve deeply into the 100 shlokas of Soundarya Lahari. The aim is to create an AI system capable of understanding, interpreting, and providing insights into the profound philosophical, spiritual, and poetic elements of the text.

Load the Dataset


In [1]:
import pandas as pd
df=pd.read_csv('/content/output.csv')
df=df.iloc[:-2]

In [2]:
df.to_csv('output.csv',index=False)

In [3]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

In [4]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files='/content/output.csv')


Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Verse', 'Instruction', 'Output'],
        num_rows: 178
    })
})

In [7]:
df.head()

,Verse,Instruction,Output
0,1.0,शिवः शक्त्या युक्तो यदि भवति शक्तः प्रभवितुंन ...,Śivaḥ - tāntric Śiva i.e. God Himself; yadi -...
1,1.0,Śivaḥ śaktyā yukto yadi bhavati śaktaḥ prabhav...,Śivaḥ - tāntric Śiva i.e. God Himself; yadi -...
2,2.0,तनीयांसं पांसुं तव चरण पङ्केरुहभवंविरिञ्चिः सं...,This verse further explains the Supremacy of Ś...
3,2.0,tanīyāṁsaṁ pāṁsuṁ tava caraṇa paṅkeruhabhavaṃv...,This verse further explains the Supremacy of Ś...
4,3.0,अविद्यानामन्तस्तिमिरमिहिरद्वीपनगरी1जडानां चैतन...,Variations in texts:1. nakarī नकरी2. bhavati भ...


Finetune Helsinki-NLP/opus-mt-mul-en model as it supports multilanguage.

Load the Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="Helsinki-NLP/opus-mt-mul-en"

tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/791k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Preprocess the dataset

In [ ]:
from datasets import concatenate_datasets
import numpy as np

tokenized_inputs = dataset.map(lambda x: tokenizer(x["Instruction"], truncation=True), batched=True)
try:
  input_lenghts =[]
  for i in tokenized_inputs['train']:
    input_lenghts.append(len(i['input_ids']))
  max_source_length = int(np.percentile(input_lenghts, 85))
except:
  pass

Map:   0%|          | 0/178 [00:00<?, ? examples/s]

In [ ]:
tokenized_targets = dataset.map(lambda x: tokenizer(x["Output"], truncation=True), batched=True)


Map:   0%|          | 0/178 [00:00<?, ? examples/s]

In [ ]:
target_lengths=[]
for i in tokenized_targets['train']:
  target_lengths.append(len(i['input_ids']))
max_target_length = int(np.percentile(target_lengths, 90))

Tokenize the dataset

In [ ]:
def preprocess_function(sample,padding="max_length"):
    inputs = ["Explain: " + item for item in sample["Instruction"]]

    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    labels = tokenizer(text_target=sample["Output"], max_length=max_target_length, padding=padding, truncation=True)


    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

tokenized_dataset["train"].save_to_disk("data/train")


Map:   0%|          | 0/178 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['Verse', 'Instruction', 'Output', 'input_ids', 'attention_mask', 'labels']


Saving the dataset (0/1 shards):   0%|          | 0/178 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from huggingface_hub import notebook_login



Load the base model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-mul-en", torch_dtype="auto")


pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Set Lora parameters

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q_proj", "v_proj"],
        # "model.encoder.layers.*.self_attn.q_proj",  # Encoder self-attention query projection
        # "model.encoder.layers.*.self_attn.v_proj",  # Encoder self-attention value projection
        # "model.decoder.layers.*.self_attn.q_proj",  # Decoder self-attention query projection
        # "model.decoder.layers.*.self_attn.v_proj",  # Decoder self-attention value projection
        # "model.decoder.layers.*.encoder_attn.q_proj",  # Decoder encoder-attention query projection
        # "model.decoder.layers.*.encoder_attn.v_proj"   # Decoder encoder-attention value projection
    # ],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# trainable params: 18874368 || all params: 11154206720 || trainable%: 0.16921300163961817

trainable params: 589,824 || all params: 78,108,672 || trainable%: 0.7551


Define the Data Collator for finetuning

In [ ]:
from transformers import DataCollatorForSeq2Seq

label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

Define the Training Args

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="output"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)
model.config.use_cache = False

Train the model

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=115, training_loss=6.329153575067934, metrics={'train_runtime': 2714.4596, 'train_samples_per_second': 0.328, 'train_steps_per_second': 0.042, 'total_flos': 24840319795200.0, 'train_loss': 6.329153575067934, 'epoch': 5.0})

In [ ]:
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)


('results/tokenizer_config.json',
 'results/special_tokens_map.json',
 'results/vocab.json',
 'results/source.spm',
 'results/target.spm',
 'results/added_tokens.json')

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-mul-en", torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # This line is the crucial change

model.eval()

print("Peft model loaded")

Peft model loaded
